In [6]:
import pandas as pd

# 1) อ่านไฟล์
df1 = pd.read_csv("36_stratagems.csv")
df2 = pd.read_csv("36strategies-description.csv")

# 2) Strip whitespace รอบ key columns
for df in (df1, df2):
    df.columns = df.columns.str.strip()
    df['ชื่อกลยุทธ์'] = df['ชื่อกลยุทธ์'].str.strip()
    df['หมวด']        = df['หมวด'].str.strip()

# 3) Aggregate df1 → list-columns
agg = (
    df1
    .groupby(['ชื่อกลยุทธ์','หมวด'], as_index=False)
    .agg({
        'ตัวอย่างสถานการณ์': lambda x: list(x),
        'วิเคราะห์':            lambda x: list(x),
    })
)
# agg now has one row per strategy, with two list-columns

# 4) Merge กับ df2
df_merged = pd.merge(
    agg,
    df2,
    on=['ชื่อกลยุทธ์','หมวด'],
    how='inner'
)
print("After merge:", df_merged.shape)   # e.g. (36, 6)
print(df_merged.head(1).to_dict())

# 5) สร้าง prompts โดย loop ผ่านแต่ละ list
analysis_prompts = []
description_prompts = []

for _, row in df_merged.iterrows():
    # (a) analysis prompts: 5 per strategy
    for sit, anl in zip(row['ตัวอย่างสถานการณ์'], row['วิเคราะห์']):
        analysis_prompts.append(
            f"กลยุทธ์: {row['ชื่อกลยุทธ์']}\n"
            f"สถานการณ์: {sit}\n"
            f"วิเคราะห์: {anl}"
        )
    # (b) description prompt: 1 per strategy
    description_prompts.append(
        f"อธิบายกลยุทธ์ \"{row['ชื่อกลยุทธ์']}\"\n\n"
        f"{row['ลักษณะ']}\n\n"
        f"ตัวอย่าง: {row['ตัวอย่างกลยุทธ์']}"
    )

# รวมกันเป็นชุดสุดท้าย
all_prompts = analysis_prompts + description_prompts
print("Total prompts:", len(all_prompts))  # = 36*5 + 36 = 216


After merge: (36, 6)
{'ชื่อกลยุทธ์': {0: 'กวนน้ำจับปลา'}, 'หมวด': {0: 'ติดพัน'}, 'ตัวอย่างสถานการณ์': {0: ['อ้วนเสี้ยวใช้ซุนจ้านบุกเกงจิ๋วแทนตัวเอง ทำให้ฮันฮกเตรียมรับไม่ทันและยึดเมืองได้', 'ฝ่ายโรงงานแพร่ข่าวข่าวลือการหยุดเดินเครื่องกระทันหัน เพื่อตรวจระบบคู่แข่งจนหยุดชะงัก', 'นักลงทุนใช้ข้อมูลสับสนเกี่ยวกับการควบกิจการเพื่อให้ตลาดราคาหุ้นผันผวนก่อนเข้าเทรด', 'ทีมแฮกเกอร์ปล่อย Trojan ปลอม เพื่อดูว่าฝ่ายป้องกันเผลอตรงไหน แล้วจึงเจาะจริง', 'ทีมส่งจดหมายข่าวผิดข้อมูลเพื่อดึงให้คู่แข่งปรับยุทธศาสตร์ไว้ผิดทางเอง']}, 'วิเคราะห์': {0: ['ใช้กองกำลังตัวแทนก่อความสับสน เปิดช่องให้เราบุกเข้ายึดพื้นที่โดยตรง', 'ปล่อยข่าวทำให้คู่แข่งตื่นตระหนก เตลิดแพลนให้เราควบคุมตลาดได้ง่ายขึ้น', 'เดือนรู้ราคาขึ้นลงเร็ว เปิดทางให้ซื้อขายได้เปรียบ', 'สร้างความปั่นป่วนเพื่อค้นหาช่องโหว่ ก่อนโจมตีหลัก', 'บอยกทางยุทธศาสตร์ของคู่พ่าย จึงสามารถบุกได้โดยไม่ติดต่อต้านหนัก']}, 'ลักษณะ': {0: 'หมายถึงการรู้จักฉวยจังหวะที่ศัตรูเกิดความปั่นป่วนภายในกองทัพให้เป็นประโยชน์ แย่งชิงเอาผลประโยชน์นั้นมาเป็นของตน นำกำลังทหารบุกเข้าโ